 --------------------------------------------------------------------------------------
# Preprocessing
### This file imports the twitter datafile and :
#####  - Creates a pandas dataframe from the data
#####  - Creates a new column with only the text of the tweet, excluding mentions and hyperlinks
#####  - Drops every tweet that does not contain any text, to remove spam links and mentions

### This result is then saved to an .xlsx file
--------------------------------------------------------------------------------------
<br>

In [1]:
# Verander dit naar je filelocation en dan werkt alles als het goed is
filelocation = "geotagged_tweets_20160812-0912.json"

## Imports

In [2]:
from pprint import pprint
import json
import pandas as pd
import re

## Data uitladen

In [3]:
%%time
tweets_data = []
tweets_file = open(filelocation, "r")

i = 0    
for line in tweets_file:
    i += 1
    #Comment de volgende twee regels uit als je alle tweets wilt
    #for i in range(2000):    
        #if i%100000 == 0:
            #print(i)
    try:
        tweet = json.loads(line)
        tweets_data.append(tweet)
    except Exception as e:
        print (e)
        continue

       
print ('We have gathered:',len(tweets_data), 'tweets.')
#print ("Information contained in a single tweet: \n")
pprint(tweets_data[0].keys())

We have gathered: 657307 tweets.
dict_keys(['created_at', 'id', 'id_str', 'text', 'source', 'truncated', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'entities', 'extended_entities', 'favorited', 'retweeted', 'possibly_sensitive', 'filter_level', 'lang', 'timestamp_ms'])
Wall time: 1min 2s


## Dataframe

In [4]:
tweets = pd.DataFrame.from_dict(tweets_data)

## Preprocessing

In [5]:
# Delete hyperlinks
#tweets['text'] = tweets['text'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)

In [8]:
def clean_string(row):
    # remove http and www links from string
    rem_hl = re.sub(r'(http\S+) | (www\S+) | (https\S+)', '', row['text'])
    
    # remove @.. from string
    rem_at_hl = re.sub(r'@(\w+) |@(\w+)', '', rem_hl)
    
    # remove hashtags #.. from string
    rem_hash_at_hl = re.sub(r'#(\w+) |@(\w+)', '', rem_at_hl)

    rem_inter = re.sub(r'[^\w\s]','', rem_hash_at_hl)
    
    # remove multiple spaces
    rem_space = re.sub(' +',' ',rem_inter)
    #print (re.search('[a-zA-Z]', rem_space))
    if (re.search('[a-zA-Z]', rem_space) == None):
        return("emptyStringRetured")
    else:
        return (rem_space)


In [9]:
tweets['CleanText'] = tweets.apply(lambda row: clean_string(row), axis=1)

In [ ]:
tweets.tail()

In [11]:
cleaned = tweets[tweets['CleanText'].str.contains("emptyStringRetured") == False]

In [12]:
len(cleaned)

610335

In [12]:
cleaned.to_json('twitter_geotagged_clean.json', orient='records', lines=True)